In [1]:
# include useful folders
import sys

In [2]:
sys.path.append("../vendors/mtl_girnet/data_prep/")

In [71]:
# enable or disable cuda
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
import json
import h5py
import numpy as np
import glob
import random
import pandas as pd
import re
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix

# nltk
import nltk

# tokenizer
from twokenize import tokenizeRawTweetText as tokenize

# for a particular dataset
from xml.dom import minidom

In [5]:
# for trying differnet types of tokenizer

# from nltk.corpus import stopwords
# from  nltk.stem import SnowballStemmer
# from tokensize_deepmoji import tokenize
# from nltk.tokenize import TweetTokenizer
# tokenizer = TweetTokenizer(a)
# from tokenizer import tokenizer
# T = tokenizer.TweetTokenizer(preserve_handles=False, preserve_hashes=False, preserve_case=False, preserve_url=False, regularize=True)
# nltk.download('stopwords')
# stop_words = stopwords.words("english")
# stemmer = SnowballStemmer("english")
# TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
# def preprocess(text, stem=False):
#     # Remove link,user and special characters
#     text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
#     tokens = []
#     for token in text.split():
#         if token not in stop_words: 
#             if stem:
#                 tokens.append(stemmer.stem(token))
#             else:
#                 tokens.append(token)
#     return tokenizer.tokenize(" ".join(tokens))

## Data Preprocessing

### Sentiment140 dataset 
https://www.kaggle.com/kazanova/sentiment140

In [6]:
# essential functions/declarations
decode_map = {0: -1, 2: 0, 4: 1}

In [7]:
#1.6 millionss tweets dataset
df = pd.read_csv('../data/'+'training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1' , names=["target", "ids", "date", "flag", "user", "text"])

In [8]:
# this is goning to take some time. chill 
df.target = df.target.apply(lambda x: decode_map[int(x)])
df.text = df.text.apply(lambda x: tokenize(x))

In [9]:
data = map( lambda x :{'sentiment': x[0] , 'tokens': x[-1] , } , df.to_numpy() )

In [10]:
en_sentiment140 = list(data)

### English-Spanish Code Mixed Data 

In [50]:
sents = {"N":-1 , "P" :1 , "NONE":0}

In [51]:
data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_train.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_train = data

In [52]:
data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_test.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_test = data

In [53]:
en_es_wssa_data = list(en_es_wssa_data_train) + list(en_es_wssa_data_test)

### Spanish Tweet Dataset

In [15]:
xmldoc = minidom.parse("../vendors/mtl_girnet/data_prep/data_cm_senti/general-tweets-train-tagged.xml")
tweets = xmldoc.getElementsByTagName('tweet')

sents = {"N":-1 , "P" :1 , "NEU":0 , 'NONE':0 , "P+" : 1 , "N+":-1 }


es_tass1_data = []

for i in range( len(tweets)-1) :
    if i == 6055:
        continue # bad jogar
    textt = tweets[i].getElementsByTagName('content')[0].childNodes[0].data
    words = tokenize( textt )
    sentiment = tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('value')[0].childNodes[0].data
    assert len(tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('entity'))==0
    es_tass1_data.append({'text':textt , 'tokens':words , 'sentiment': sents[sentiment] })

### Some english tweet data

In [16]:
data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/twitter4242.txt", "r", encoding="utf-8",errors='ignore').read().split("\n")[1:-1]
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

en_twitter_data = list(data)

### es2_twitter_data

In [17]:
data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_dev_complete.txt", encoding="utf-8").read().split("\n")[1:-1]
data += open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_test_average_complete.tsv", encoding="utf-8").read().split("\n")[1:-2]

data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

es2_twitter_data = list(data)

In [18]:
print("Code-Mixed: en_es_wssa_data: %d" % len(en_es_wssa_data))
print("Spanish: es2_twitter_data: %d" % len(es2_twitter_data))
print("Spanish: es_tass1_data: %d" % len(es_tass1_data))
print("English: en_twitter_data: %d" % len(en_twitter_data))
print("English: en_sentiment140: %d" %len(en_sentiment140))

Code-Mixed: en_es_wssa_data: 3062
Spanish: es2_twitter_data: 3202
Spanish: es_tass1_data: 7217
English: en_twitter_data: 4241
English: en_sentiment140: 1600000


# For LASER: To get embeddings for sentences

In [61]:
def write_sentences(data, name):
    with open(name,"w") as f:
        temp = ""
        for row in data:
            temp = temp + str(row['text']).strip() + "\n"
        f.write(temp)

def write_sentences_sep(data, name):
    with open(name,"w") as f:
        temp = []
        for row in data:
            temp.append(str(" ".join(row['tokens'])).strip() + "\n")
    return temp

In [62]:
write_sentences(en_es_wssa_data,"en_es.txt")
write_sentences(es2_twitter_data,"es2_twitter.txt")
write_sentences(es_tass1_data,"es_tass1.txt")
write_sentences(en_twitter_data, "en_twiter.txt")

In [63]:
# PRLEASE DON'T EXECUTE THIS MOTHERFUCKER
lines_ = write_sentences_sep(en_sentiment140, "en_sentiment140.txt")
with open("en_sentiment140.txt", "w") as f:
    for line in lines_:
        f.write(line)

In [24]:
def get_y(data):
    from keras.utils import to_categorical
    y = []
    for row in data:
        y.append(int(row['sentiment']))
    y = to_categorical(y,num_classes=3)
    return y

In [68]:
en_es_y =  get_y(en_es_wssa_data)
es_twitter_y = get_y(es2_twitter_data)
es_tass_y = get_y(es_tass1_data)
en_twitter_y = get_y(en_twitter_data)
en_sentiment140_y = get_y(en_sentiment140)

# For LASER: Load Embedding

In [26]:
def load(file):
    dim = 1024
    X = np.fromfile(file, dtype=np.float32, count=-1)                                                                          
    X.resize(X.shape[0] // dim, dim) 
    print(file + ": " + str(X.shape))
    return X

In [69]:
en_es_emb = load("en_es.raw")
es_twitter_emb =load("es2_twitter.raw")
es_tass_emb =load("es_tass.taw")
en_twitter_emb =load("en_twiter.raw")
en_sentiment140_emb = load("en_sentiment140.raw")

en_es.raw: (3062, 1024)
es2_twitter.raw: (3202, 1024)
es_tass.taw: (7219, 1024)
en_twiter.raw: (4241, 1024)
en_sentiment140.raw: (1600000, 1024)


In [67]:
es_tass_y.shape

(7217, 3)

# For LASER: Model

In [28]:
from keras.layers import *
from keras.models import Sequential

In [94]:
model = Sequential()
model.add(Dense(512, input_shape=(1024,)))
# # model.add(Dropout(0.4))
# model.add(Dense(512))
model.add(Dropout(0.2))
model.add(Dense(3 , activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy', f1])

In [95]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 1539      
Total params: 526,339
Trainable params: 526,339
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(en_sentiment140_emb, en_sentiment140, batch_size=128, epochs=15, validation_split=0.4)

Train on 960000 samples, validate on 640000 samples
Epoch 1/15
268032/960000 [=======>......................] - ETA: 52s - loss: 0.3325 - acc: 0.8641 - f1: 0.8636

In [81]:
model.fit(en_twitter_emb, en_twitter_y, batch_size=32, epochs=15, validation_split=0.1)

Train on 3816 samples, validate on 425 samples
Epoch 1/15
3816/3816 [==============================] - 1s 287us/step - loss: 0.7539 - acc: 0.7078 - f1: 0.7012 - val_loss: 0.8980 - val_acc: 0.6447 - val_f1: 0.6431
Epoch 2/15
3816/3816 [==============================] - 1s 290us/step - loss: 0.5397 - acc: 0.7707 - f1: 0.7691 - val_loss: 1.0082 - val_acc: 0.6071 - val_f1: 0.6129
Epoch 3/15
3816/3816 [==============================] - 1s 303us/step - loss: 0.5219 - acc: 0.7849 - f1: 0.7799 - val_loss: 0.9984 - val_acc: 0.6165 - val_f1: 0.6155
Epoch 4/15
3816/3816 [==============================] - 1s 307us/step - loss: 0.5238 - acc: 0.7786 - f1: 0.7744 - val_loss: 1.1235 - val_acc: 0.5906 - val_f1: 0.6002
Epoch 5/15
3816/3816 [==============================] - 1s 334us/step - loss: 0.5151 - acc: 0.7830 - f1: 0.7802 - val_loss: 1.0336 - val_acc: 0.6259 - val_f1: 0.6200
Epoch 6/15
3816/3816 [==============================] - 1s 317us/step - loss: 0.5144 - acc: 0.7911 - f1: 0.7862 - val_loss:

In [82]:
model.evaluate(es_twitter_emb, es_twitter_y)

3202/3202 [==============================] - 0s 78us/step


[1.4272800997448742, 0.5137414116177389, 0.506156767367721]

In [86]:
model.fit(es_twitter_emb, es_twitter_y, batch_size=32, epochs=20, validation_split=0.1)

Train on 2881 samples, validate on 321 samples
Epoch 1/20
2881/2881 [==============================] - 1s 317us/step - loss: 0.5167 - acc: 0.7921 - f1: 0.7843 - val_loss: 1.3556 - val_acc: 0.5016 - val_f1: 0.4973
Epoch 2/20
2881/2881 [==============================] - 1s 318us/step - loss: 0.5185 - acc: 0.7904 - f1: 0.7851 - val_loss: 1.3300 - val_acc: 0.5452 - val_f1: 0.5385
Epoch 3/20
2881/2881 [==============================] - 1s 332us/step - loss: 0.5101 - acc: 0.7886 - f1: 0.7847 - val_loss: 1.2849 - val_acc: 0.5483 - val_f1: 0.5510
Epoch 4/20
2881/2881 [==============================] - 1s 317us/step - loss: 0.4923 - acc: 0.8028 - f1: 0.8028 - val_loss: 1.2878 - val_acc: 0.5389 - val_f1: 0.5429
Epoch 5/20
2881/2881 [==============================] - 1s 315us/step - loss: 0.5024 - acc: 0.7886 - f1: 0.7822 - val_loss: 1.2795 - val_acc: 0.5701 - val_f1: 0.5655
Epoch 6/20
2881/2881 [==============================] - 1s 307us/step - loss: 0.4831 - acc: 0.7990 - f1: 0.7971 - val_loss:

In [87]:
model.evaluate(en_es_emb,en_es_y)

3062/3062 [==============================] - 0s 73us/step


[1.9322782474588833, 0.4866100587072441, 0.4831954719271246]

# Load Embedding

In [21]:
import io
import numpy as np

In [22]:
## essential functions
def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [23]:
## NEED TO RUN MUSE BEFORE THIS and to get this path
src_path = '../vendors//MUSE/dumped/debug/4u9hakomha/vectors-en.txt'
tgt_path = '../vendors//MUSE/dumped/debug/4u9hakomha/vectors-es.txt'
nmax = 100000  # maximum number of word embeddings to load

src_embeddings, src_id2word, src_word2id = load_vec(src_path, nmax)
tgt_embeddings, tgt_id2word, tgt_word2id = load_vec(tgt_path, nmax)

In [24]:
src_embeddings, src_id2word, src_word2id = load_vec(src_path, nmax)
tgt_embeddings, tgt_id2word, tgt_word2id = load_vec(tgt_path, nmax)

# Analysis embedding

In [25]:
def get_nn(word, src_emb, src_id2word, tgt_emb, tgt_id2word, K=5):
    print("Nearest neighbors of \"%s\":" % word)
    word2id = {v: k for k, v in src_id2word.items()}
    word_emb = src_emb[word2id[word]]
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [26]:
# printing nearest neighbors in the source space
src_word = 'cat'
get_nn(src_word, src_embeddings, src_id2word, src_embeddings, src_id2word, K=5)

Nearest neighbors of "cat":
1.0000 - cat
0.7322 - cats
0.6453 - kitten
0.6381 - dog
0.6218 - kittens


In [27]:
# printing nearest neighbors in the target space
src_word = 'cat'
get_nn(src_word, src_embeddings, src_id2word, tgt_embeddings, tgt_id2word, K=5)

Nearest neighbors of "cat":
0.6266 - gato
0.5317 - perro
0.5213 - gatito
0.4872 - gorila
0.4767 - ratoncito


In [28]:
src_embeddings.shape

(100000, 300)

# Merge Embedding

In [53]:
from keras.preprocessing import sequence
from keras.utils import to_categorical

In [30]:
def merge_embeddings(src_embeddings, tgt_embeddings):
    
    # make combined embedding mattrix
    embedding_matrix = src_embeddings.copy().tolist()
    embedding_matrix.extend(tgt_embeddings.tolist())
    embedding_matrix = np.array(embedding_matrix)
    
    # make combined id2word and word2id
    id2word = src_id2word.copy()
    word2id = src_word2id.copy()
    
    next_id = len(id2word.keys())
    counter = len(id2word.keys())
    
    to_be_removed_id = []
    common_words = []
    
    for key in tgt_id2word:
        if tgt_id2word[key] in word2id:
            to_be_removed_id.append(counter)
            common_words.append(tgt_id2word[key])
            embedding_matrix[word2id[tgt_id2word[key]]] =  (embedding_matrix[word2id[tgt_id2word[key]]] + embedding_matrix[counter])/2
        else:
            id2word[next_id] = tgt_id2word[key]
            word2id[tgt_id2word[key]] = next_id
            next_id += 1
        counter += 1
        
    embedding_matrix = np.delete(embedding_matrix, to_be_removed_id, axis=0)
        
    return embedding_matrix, id2word, word2id, common_words

In [31]:
embedding_matrix, id2word, word2id, common_words = merge_embeddings(src_embeddings, tgt_embeddings)

In [32]:
print("embedding size: ", str(embedding_matrix.shape))

embedding size:  (161832, 300)


In [33]:
print("Number of common words in both the embedding %d" % len(common_words))

Number of common words in both the embedding 38168


In [34]:
# ADD UNK
# TODO:

# Data Preprocess Part 2

In [35]:
from  nltk.stem import SnowballStemmer

In [36]:
MAX_SEQUENCE_LENGTH = 20

In [37]:
words_not_found = 0
def from_datas_to_x_y(list_data, word2id, max_seq_len=20, max_classes=3, seed=0):
    stemmer = SnowballStemmer("english")  
    words_not_found = 0
    def to_x(sample):
        global words_not_found
        x = []
        for word in sample['tokens']:
            # lower the word 
            word = word.lower()
            if word in word2id:
                x.append(word2id[word]) 
            else:
                stem = stemmer.stem(word) # find stem
                if stem in word2id:
                    x.append(word2id[stem])
                else:
                    words_not_found = words_not_found + 1
                    pass
        return x

    def to_x_y(data):
        temp =  np.array(list(map(lambda x : [to_x(x), x['sentiment']], data)))
        x = list(sequence.pad_sequences(temp[:,0], maxlen=max_seq_len))
        y = list(to_categorical(temp[:,1],num_classes=max_classes))
        return x, y
    
    x,y = [],[]
    for data in list_data:
        x_, y_ = to_x_y(data)
        print("x: %d \t y: %d" % (len(x_),len(y_)))
        x.extend(x_)
        y.extend(y_)
    x = np.array(x)
    y = np.array(y)
    x,y = shuffle(x, y, replace=True)
    
    print("Not Found words = %f" % (float(words_not_found)/(x.shape[0])))
    return x,y

In [71]:
# x_test,y_test = from_datas_to_x_y([en_es_wssa_data],word2id)

In [83]:
def evaluate_one(model, x, y):
    y_pred = model.predict(x,verbose=0)
    print("Classification Report")
    print(classification_report(np.argmax(y, axis=1), np.argmax(y_pred,axis=1)))
    print("Confusion Matrix")
    print(confusion_matrix(np.argmax(y, axis=1), np.argmax(y_pred,axis=1)))
def evaluate(model):
    print("Code Mixed: ")
    x_test,y_test = from_datas_to_x_y([en_es_wssa_data],word2id)
    evaluate_one(model, x_test, y_test)
    
    print("\nEnglish: ")
    x_test,y_test = from_datas_to_x_y([en_twitter_data],word2id)
    evaluate_one(model, x_test, y_test)
    
    print("\nSpanish: 1")
    x_test,y_test = from_datas_to_x_y([es2_twitter_data],word2id)
    evaluate_one(model, x_test, y_test)
    
    print("\nSpanish: 2")
    x_test,y_test = from_datas_to_x_y([es_tass1_data],word2id)
    evaluate_one(model, x_test, y_test)

In [73]:
# print(x.shape)
# print(y.shape)

# Base Model

In [41]:
from keras.layers import *
from keras.models import Sequential

In [42]:
#embedding
EMBEDDING_DIM = embedding_matrix.shape[1] 
MAX_NUM_WORDS = embedding_matrix.shape[0]

In [43]:
# for cnn_lstm

# Convolution
kernel_size = 5
filters = 64
pool_size = 1
# lstm
lstm_output_size = 70

In [101]:
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS,
                     EMBEDDING_DIM,
                     weights=[embedding_matrix],
                     input_length=MAX_SEQUENCE_LENGTH,
                     trainable=True))
# model.add(Dropout(0.2))
# model.add(Conv1D(filters,
#                  kernel_size,
#                  activation='relu',
#                  strides=1))
# model.add(MaxPooling1D(pool_size=pool_size))
# model.add(LSTM(lstm_output_size))
model.add(Bidirectional(LSTM(512, dropout=0.1, recurrent_dropout=0.1)))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1])

In [102]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 300)           48549600  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 1024)              3330048   
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 3075      
_________________________________________________________________
activation_5 (Activation)    (None, 3)                 0         
Total params: 51,882,723
Trainable params: 51,882,723
Non-trainable params: 0
_________________________________________________________________


In [103]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(monitor='val_f1', patience=4, restore_best_weights=True, verbose=5)
checkpointer = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_f1:.2f}.hdf5', verbose=1, save_best_only=True, save_weights_only=True, monitor='val_f1')

# Just english

In [50]:
x_train, y_train = from_datas_to_x_y([en_sentiment140,en_twitter_data],word2id)

x: 1600000 	 y: 1600000
x: 4241 	 y: 4241
Not Found words = 0.000000


In [104]:
get_class_weight(y_train)

{0: 273.9482581967213, 1: 0.6673159957071905, 2: 0.6676417599622448}

In [126]:
history = model.fit(x_train , y_train, 
                    epochs=5, 
                    batch_size=648,
                    validation_split=0.3, 
                    shuffle=True)

Train on 1122968 samples, validate on 481273 samples
Epoch 1/5
1122968/1122968 [==============================] - 687s 612us/step - loss: 0.2604 - acc: 0.8884 - f1: 0.8883 - val_loss: 0.5404 - val_acc: 0.8080 - val_f1: 0.8080
Epoch 2/5
 205416/1122968 [====>.........................] - ETA: 8:44 - loss: 0.2025 - acc: 0.9153 - f1: 0.9153

KeyboardInterrupt: 

In [127]:
evaluate(model)

Code Mixed: 
x: 3062 	 y: 3062
Not Found words = 0.000000
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.00      0.00      1313
           1       0.41      0.69      0.52       963
           2       0.37      0.68      0.48       786

   micro avg       0.39      0.39      0.39      3062
   macro avg       0.59      0.46      0.33      3062
weighted avg       0.65      0.39      0.29      3062

Confusion Matrix
[[  3 687 623]
 [  0 660 303]
 [  0 252 534]]

English: 
x: 4241 	 y: 4241
Not Found words = 0.000000
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.55      0.71      1952
           1       0.65      0.96      0.78      1340
           2       0.75      0.94      0.83       949

   micro avg       0.77      0.77      0.77      4241
   macro avg       0.80      0.82      0.77      4241
weighted avg       0.83      0.77      0.76      4241

Confusion Matrix


# Just Spanish

In [111]:
x_train_es, y_train_es =  from_datas_to_x_y([es_tass1_data, es2_twitter_data], word2id=word2id)

x: 7217 	 y: 7217
x: 3202 	 y: 3202
Not Found words = 0.000000


In [112]:
get_class_weight(y_train_es)

{0: 1.0385765550239234, 1: 0.8748110831234257, 2: 1.1185185185185185}

In [119]:
history = model.fit(x_train_es, y_train_es, 
                    epochs=10, 
                    batch_size=128, 
                    validation_split=0.1, 
                    shuffle=True)

Train on 9377 samples, validate on 1042 samples
Epoch 1/10
9377/9377 [==============================] - 10s 1ms/step - loss: 0.3096 - acc: 0.8860 - f1: 0.8843 - val_loss: 0.4082 - val_acc: 0.8369 - val_f1: 0.8290
Epoch 2/10
9377/9377 [==============================] - 10s 1ms/step - loss: 0.1528 - acc: 0.9469 - f1: 0.9464 - val_loss: 0.4552 - val_acc: 0.8340 - val_f1: 0.8333
Epoch 3/10
9377/9377 [==============================] - 11s 1ms/step - loss: 0.0749 - acc: 0.9752 - f1: 0.9760 - val_loss: 0.5621 - val_acc: 0.8215 - val_f1: 0.8221
Epoch 4/10
9377/9377 [==============================] - 10s 1ms/step - loss: 0.0491 - acc: 0.9841 - f1: 0.9843 - val_loss: 0.6701 - val_acc: 0.8148 - val_f1: 0.8139
Epoch 5/10
1152/9377 [==>...........................] - ETA: 8s - loss: 0.0415 - acc: 0.9887 - f1: 0.9878

KeyboardInterrupt: 

In [120]:
evaluate(model)

Code Mixed: 
x: 3062 	 y: 3062
Not Found words = 0.000000
Classification Report
              precision    recall  f1-score   support

           0       0.53      0.36      0.43      1313
           1       0.50      0.58      0.53       963
           2       0.45      0.60      0.52       786

   micro avg       0.49      0.49      0.49      3062
   macro avg       0.49      0.51      0.49      3062
weighted avg       0.50      0.49      0.49      3062

Confusion Matrix
[[478 417 418]
 [256 554 153]
 [170 144 472]]

English: 
x: 4241 	 y: 4241
Not Found words = 0.000000
Classification Report
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1952
           1       0.90      0.89      0.89      1340
           2       0.89      0.85      0.87       949

   micro avg       0.89      0.89      0.89      4241
   macro avg       0.89      0.89      0.89      4241
weighted avg       0.89      0.89      0.89      4241

Confusion Matrix


## Both English and Spanish

In [128]:
x_train_both, y_train_both = from_datas_to_x_y([en_twitter_data,es2_twitter_data,es_tass1_data],word2id)

x: 4241 	 y: 4241
x: 3202 	 y: 3202
x: 7217 	 y: 7217
Not Found words = 0.000000


In [129]:
x_test,y_test = from_datas_to_x_y([en_es_wssa_data],word2id)

x: 3062 	 y: 3062
Not Found words = 0.000000


In [130]:
model.fit(x_train_both,y_train_both,
          validation_data=(x_test,y_test),
          batch_size=128,
          epochs=1,
          shuffle=True)

Train on 14660 samples, validate on 3062 samples
Epoch 1/1
14660/14660 [==============================] - 17s 1ms/step - loss: 0.1047 - acc: 0.9641 - f1: 0.9636 - val_loss: 2.3280 - val_acc: 0.4700 - val_f1: 0.4646


In [131]:
evaluate(model)

Code Mixed: 
x: 3062 	 y: 3062
Not Found words = 0.000000
Classification Report
              precision    recall  f1-score   support

           0       0.51      0.31      0.38      1313
           1       0.48      0.58      0.53       963
           2       0.43      0.61      0.50       786

   micro avg       0.47      0.47      0.47      3062
   macro avg       0.47      0.50      0.47      3062
weighted avg       0.48      0.47      0.46      3062

Confusion Matrix
[[403 445 465]
 [224 555 184]
 [156 149 481]]

English: 
x: 4241 	 y: 4241
Not Found words = 0.000000
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1952
           1       1.00      1.00      1.00      1340
           2       1.00      1.00      1.00       949

   micro avg       1.00      1.00      1.00      4241
   macro avg       1.00      1.00      1.00      4241
weighted avg       1.00      1.00      1.00      4241

Confusion Matrix


# Cross-Lingual

In [30]:
from keras import backend as K


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [31]:
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
def get_class_weight(y):
    """
    Used from: https://stackoverflow.com/a/50695814
    TODO: check validity and 'balanced' option
    :param y: A list of one-hot-encoding labels [[0,0,1,0],[0,0,0,1],..]
    :return: class-weights to be used by keras model.fit(.. class_weight="") -> {0:0.52134, 1:1.adas..}
    """
    y_integers = np.argmax(y, axis=1)
    class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
    d_class_weights = dict(enumerate(class_weights))
    return d_class_weights

In [ ]:
# doesn't work

In [29]:
from dlblocks import text
from dlblocks.pyutils import mapArrays , loadJson , saveJson , selectKeys , oneHotVec , padList
from dlblocks.pyutils import int64Arr , floatArr

In [ ]:
vocab = text.Vocabulary()

for d in es_tass1_data + en_es_wssa_data + en_twitter_data + es2_twitter_data :
    vocab.add_words( d['tokens']  )

    
vocab.keepTopK(25000)



maxSentenceL = 150

def vecc( d ):
    ret = {}
    words   = d['tokens']
    wordids = map( vocab , words )
    ret['sentence'] = int64Arr( padList( wordids , maxSentenceL , 0 , 'left') )
    ret['sentiment_val'] =  floatArr( d['sentiment'] )
    ret['sentiment_id'] =  int64Arr( d['sentiment'] + 1 )
    ret['sentiment_onehot'] =  floatArr( oneHotVec( d['sentiment']+1 , 3  ) )

    return ret





en_es_wssa_data_train_arr = mapArrays( en_es_wssa_data_train , vecc )
en_es_wssa_data_test_arr = mapArrays( en_es_wssa_data_test , vecc )

en_twitter_data_train_arr = mapArrays( en_twitter_data , vecc )
es_tass1_datatrain_arr = mapArrays( es_tass1_data , vecc )

datasets = {"en_es_wssa_data_train_arr":en_es_wssa_data_train_arr ,
           "en_es_wssa_data_test_arr":en_es_wssa_data_test_arr ,
           "en_twitter_data_train_arr":en_twitter_data_train_arr ,
           "es_tass1_datatrain_arr": es_tass1_datatrain_arr }



outFNN = "../data/senti_prepped.h5"

f = h5py.File(outFNN , "w")
for kk in datasets.keys():
    f.create_group( kk  )
    for k in datasets[kk].keys():
        f[ kk ].create_dataset( k , data=datasets[kk][k] )

print "HDF5 file created !"